In [8]:
from itertools import combinations
import numpy as np
import scipy.optimize as opt
import tifffile as tf
import matplotlib.pyplot as plt
from sklearn import linear_model 

def interploate_E(refs, e):
    n = np.shape(refs)[1]
    refs = np.array(refs)
    ref_e = refs[:, 0]
    ref = refs[:, 1:n]
    all_ref = []
    for i in range(n - 1):
        ref_i = np.interp(e, ref_e, ref[:, i])
        all_ref.append(ref_i)
    return np.array(all_ref)

def xanes_fitting(im_stack, e_list, refs, method='NNLS'):
    """Linear combination fit of image data with reference standards"""
    en,im1,im2 = np.shape(im_stack)

    int_refs = (interploate_E(refs, e_list))
    im_array = im_stack.reshape(en, im1*im2)

    if method == 'NNLS':
        
        coeffs_arr = []
        r_factor_arr = []
        
        for i in range(im1*im2):
            coeffs, r = opt.nnls(int_refs.T, im_array[:,i])
            coeffs_arr.append(coeffs)
            r_factor_arr.append(r)
            
        abundance_map = np.reshape(coeffs_arr,(im1,im2,-1))
        r_factor = np.reshape(r_factor_arr,(im1,im2))
        
        
    elif method == 'LASSO':

        coeffs_arr = []
        r_factor_arr = []
        reg = linear_model.Ridge(alpha=0.1)
        for i in range(im1 * im2):
            fit_results = reg.fit(int_refs.T, im_array[:, i])
            r = fit_results.score(int_refs.T, im_array[:, i])
            coeffs_arr.append(fit_results.coef_)
            r_factor_arr.append(r)

        abundance_map = np.reshape(coeffs_arr, (im1, im2, -1))
        r_factor = np.reshape(r_factor_arr, (im1, im2))

    return abundance_map,r_factor

In [9]:
img_stack = tf.imread(r'C:\Users\pattammattel\Desktop\MIDAS_Admin\sample_data\Site4um.tiff')
e_list = np.loadtxt(r'C:\Users\pattammattel\Desktop\MIDAS_Admin\sample_data\Site4um.txt')
refs = np.loadtxt(r'C:\Users\pattammattel\Desktop\MIDAS_Admin\sample_data\test_ref_athena.nor')

In [10]:
sample_spec = img_stack.mean(1).mean(1)
int_refs = interploate_E(refs, e_list)
en,im1,im2 = np.shape(img_stack)
im_array = img_stack.reshape(en, im1*im2)

In [11]:
reg = linear_model.Lasso(alpha=0.001, positive=True)
fit_ = reg.fit(int_refs.T, sample_spec)

In [12]:
fit = fit_.coef_@int_refs
r_factor = (sample_spec.sum(-1)-fit.sum(-1))/sample_spec.sum(-1)
r_factor

0.12049866184979328

In [15]:
y_mean = np.sum(sample_spec)/len(sample_spec)
SS_tot = np.sum((sample_spec-y_mean)**2)
SS_res = np.sum((sample_spec - fit)**2)
r_square = 1 - (SS_res/ SS_tot)
r_square

0.8929329569618139